In [1]:
import random
from collections import deque
import torch
from env import HighwayEnv, convert_highd_sample_to_gail_expert
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from buffer import HighwayEnvMemoryBuffer

In [2]:
buffer = HighwayEnvMemoryBuffer(300)

expert_data, df = convert_highd_sample_to_gail_expert(
    sample_csv=r"./data/26_sample_tracks.csv",
    meta_csv=r"E:\Data\highd-dataset-v1.0\data\26_recordingMeta.csv",
    forward=False,
    p_agent=0.90
)

In [3]:
NUM_STEPS = 300

In [4]:
# Create the environment and set expert data.
env = HighwayEnv(dt=0.2, T=50, generation_mode=False, demo_mode=True)
# Uncomment and update the following line when expert_data is available:
env.set_expert_data(expert_data)


In [5]:
# 
obs = env.reset() 
#
for step in trange(NUM_STEPS):
    # For demonstration, sample random actions for each vehicle slot.
    # Action shape: (N_max, 2). Since have set DEMO_MODE=TRUE, the actions will be overwritten by the agents
    action = torch.full((env.N_max, 2), 0.0)
    # Step the environment: we get new observation, reward, done, and info.
    next_obs, reward, done, info = env.step(action)

    buffer.push(obs, action, reward, next_obs, done)


100%|██████████| 300/300 [00:04<00:00, 70.14it/s]


In [6]:
obs, action, reward, next_obs, done = buffer.sample(64)

In [9]:
obs[0].shape

torch.Size([64, 50, 100, 7])

In [12]:
obs[0][52,-1][...,-1]

tensor([ 5.3700,  5.3700,  5.3700,  5.3700,  5.3700,  9.1250,  9.1250, 12.9250,
         5.3700,  9.1250, 12.9250,  9.1250,  9.1250, 12.9250,  5.3700, 12.9250,
         5.3700, 12.9250,  9.1250,  5.3700, 12.9250,  9.1250, 12.9250, 12.9250,
         5.3700,  9.1250, 12.9250,  5.3700, 12.9250,  9.1250,  5.3700, 12.9250,
         9.1250, 12.9250,  9.1250,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     nan,     nan,     nan,     nan,     nan,
            nan,     nan,     nan,     n

In [7]:
batch = random.sample(buffer.buffer, 64)
states, actions, rewards, next_states, dones = zip(*batch)

In [11]:
state_keys=['time_dependent','time_independent', 'lane_markers', 'boundary_lines', 'agent_mask']

In [17]:
[ torch.stack([obs[key] for obs in states]) for key in state_keys ][2].shape

torch.Size([64, 2])

In [ ]:
actions

AttributeError: 'tuple' object has no attribute 'shape'